In [163]:
import csv
import torch
fileName = 'data/loan_data.csv'

with open(fileName) as csvFile:
    reader = csv.reader(csvFile)
    variables = next(reader)
    columnInfo = [[] for _ in range(len(variables))]
    data = torch.tensor([])
    for row in reader:
        t = torch.zeros(1, len(variables))
        for columnIndex, column in enumerate(row):
            try:
                t[0][columnIndex] = float(column)
                t[0][columnIndex] = int(column)
            except ValueError:
                if column not in columnInfo[columnIndex]:
                    columnInfo[columnIndex].append(column)
                t[0][columnIndex] = columnInfo[columnIndex].index(column)
        data = torch.cat((data, t))
data.shape

torch.Size([45000, 14])

In [164]:
import pandas as pd
df = pd.DataFrame(data)
df = df.set_axis(variables, axis=1)
df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,561.0,0.0,1.0
1,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,504.0,1.0,0.0
2,2.0,0.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,0.0,635.0,0.0,1.0
3,3.0,0.0,2.0,3.0,0.0,0.0,0.0,2.0,3.0,2.0,1.0,675.0,0.0,1.0
4,4.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,4.0,3.0,2.0,586.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,16.0,1.0,3.0,8721.0,6.0,0.0,58.0,2.0,906.0,14.0,0.0,645.0,0.0,1.0
44996,22.0,0.0,3.0,32251.0,17.0,0.0,60.0,4.0,317.0,42.0,9.0,621.0,0.0,1.0
44997,12.0,1.0,3.0,33987.0,7.0,0.0,4481.0,5.0,666.0,12.0,7.0,668.0,0.0,1.0
44998,11.0,1.0,2.0,33988.0,4.0,0.0,26.0,1.0,198.0,48.0,5.0,604.0,0.0,1.0
